In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType

customers_schema = StructType([
    StructField("customer_id", IntegerType()),
    StructField("customer_name", StringType()),
    StructField("date_of_birth", DateType()),
    StructField("telephone", StringType()),
    StructField("email", StringType()),
    StructField("member_since", DateType()),
    StructField("created_timestamp", TimestampType())])


In [0]:
customers_df = (spark.readStream
    .format("json")
    .schema(customers_schema)
    .load("/Volumes/gizmobox/landing/operational_data/customers_stream"))

In [0]:
from pyspark.sql.functions import col, current_timestamp

customers_transformed_df = (customers_df.withColumn("file_path", col("_metadata.file_path"))
            .withColumn("ingestion_date", current_timestamp()))

In [0]:
streaming_query = customers_transformed_df.writeStream.format("delta").option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream").table("gizmobox.bronze.customers_stream")